# Data Transformation Diversity and Inclusivity

Dans cette partie nous allons formater la table que nous allons utiliser pour nos algorithmes.
Dans un premier temps nous choississons comme objectif de prédire les recettes d'un film.
On va donc prendre en compte toutes les données relatives au travail en amont de la sortie et donc sortir les donnnées issues des utilisateurs comme par exemple la popularité.
Dans un deuxième temps on va essayer de tester une hypothèse sur ces revenues :
Est-ce que la diversité (sexe, nationnalité) influence ces revenus ?

In [ ]:
# Libraries import
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter

In [ ]:
# Dataset import

credits = pd.read_csv('./archive/credits.csv', delimiter=',',index_col="id")
# keywords = pd.read_csv('./archive/keywords.csv', delimiter=',',index_col="id") # Pas très informatif pour notre question
movies = pd.read_csv('./archive/movies_metadata.csv', delimiter=',').\
                     drop(['belongs_to_collection', 'homepage', 'imdb_id', 'poster_path', 'status', 'title', 'video'], axis=1).\
                     drop([19730, 29503, 35587]) 

In [ ]:
movies['id'] = movies['id'].astype('int64') # incorrect datatype for merge 
df = movies.merge(credits, on='id')

In [ ]:
# Converssion Json format en nominal format
def get_dictionary(s):
    try:
        d = eval(s)
    except:
        d = {}
    return d

Pour obtenir cette notion de diversité, on peut ce baser sur le genre des personnes et leur ordre d'importance.
Le genre est associé au personnage et nom à l'acteur. (2 = homme, 1 = femme)
Pour l'équipe de tournage, on a accès au genre mais connaitre la position dans l'organigramme et donc son influence est plus compliqué.

Après avoir analyser les données il est apparu qu'il n'y avait pas de différence entre le genre du charactère et le genre de l'acteur

In [ ]:
# df.keywords = df.keywords.map(lambda x: [d['name'] for d in get_dictionary(x)]).map(lambda x: ','.join(map(str, x)))
df.genres = df.genres.map(lambda x: [d['name'] for d in get_dictionary(x)]).map(lambda x: ','.join(map(str, x)))
# df.production_companies = df.production_companies.map(lambda x: [d['name'] for d in get_dictionary(x)]).map(lambda x: ','.join(map(str, x))) # trop de valeur manquantes
df.spoken_languages = df.spoken_languages.map(lambda x: [d['name'] for d in get_dictionary(x)]).map(lambda x: ','.join(map(str, x)))
# df.production_countries = df.production_countries.map(lambda x: [d['name'] for d in get_dictionary(x)]).map(lambda x: ','.join(map(str, x)))

# New columns
# df['characters'] = df.cast.map(lambda x: [d['character'] for d in get_dictionary(x)]).map(lambda x: ','.join(map(str, x)))
df['characters_gender'] = df.cast.map(lambda x: [d['gender'] for d in get_dictionary(x)]).map(lambda x: ','.join(map(str, x)))
# df['actors'] = df.cast.map(lambda x: [d['name'] for d in get_dictionary(x)]).map(lambda x: ','.join(map(str, x)))
# df['actors_order'] = df.cast.map(lambda x: [d['order'] for d in get_dictionary(x)]).map(lambda x: ','.join(map(str, x)))
# df.crew = df.crew.map(lambda x: [d['name'] for d in get_dictionary(x)]).map(lambda x: ','.join(map(str, x)))
df['crew_gender'] = df.crew.map(lambda x: [d['gender'] for d in get_dictionary(x)]).map(lambda x: ','.join(map(str, x)))


In [ ]:
df.pop('production_companies')
df.pop('production_countries')
df.pop('cast')
df.pop('crew')
df.pop('original_title')
df.pop('popularity')
df.pop('tagline')
df.pop('vote_average')
df.pop('vote_count')
df.pop('overview')
df.pop('id')

In [ ]:
df.head()

In [ ]:
language_list = [i for i in df.original_language]
print(Counter(language_list))
print(len(Counter(language_list))) 

Il y a trop de langues pour que cela soit pertinant à transcrire en onehotencoding. On peut cependant noter que la plupart des langues sont peu présentes.
On va donc réduire l'ensemble des langues utilisés à 'en' et 'other'. Si la puissance de calcul est suifisant on pourrait voir à rajouter d'autres langues.

0 = other,
1 = english

Les valeurs manquantes seront remplacé par de l'anglais (valeur la plus probable)

In [ ]:
df.original_language = df.original_language.fillna(1)
df.original_language = df.original_language.replace(to_replace=r'(^((?!en).)*$)', value=0, regex=True)
df.original_language = df.original_language.replace(to_replace=r'en', value=1, regex=True)

In [ ]:
genres_list = []
for i in df['genres']:
    genres_list.extend(i.split(','))
print(Counter(genres_list))
print(len(Counter(genres_list))) # 21 genres possibilité de faire du onehotencoding mais rajout de beaucoup de colonnes

Il y a 21 genres de film. Comme vu dans le notebook de visualisation 50% des films sont dans [drama, comedy, thriller, romance].
Néanmoins, il n'y a pas de marche significative dans les genres après pour pouvoir simplifier les colonnes.
Pour le moment on ne va pas réduire le nombre de colonnes.

In [ ]:
# companies_list = [i for i in df.production_companies]
# print(Counter(companies_list))
# print(len(Counter(companies_list))) 
# Beaucoup de valeur manquantes 

Beaucoup de valeurs manquantes (12000) donc on va supprimer cette colonne.

In [ ]:
# companies_list = [i for i in df.production_countries]
# print(Counter(companies_list))
# print(len(Counter(companies_list))) 
# Beaucoup de valeur manquantes mais large

Beaucoup de valeur manquantes (6500). Dans la mesure où on a déjà une information quant à la langue d'origine. On peut supposer que cette colonne ajoute que peut d'informations supplémentaires (surtout si on a décidé de séparer en 'english' et 'other'). On ne va donc pas garder cette colonne. Même si il y est hautement probable que le pays d'origine influence le revenu.

In [ ]:
language_list = []
for i in df['spoken_languages']:
    language_list.extend(i.split(','))
print(Counter(language_list))
print(len(Counter(language_list)))

Il y a 75 langues différentes. On va procéder au même tri que précédemment, i.e les films en anglais versus les autres.
Possibilité aussi de rendre compte de la diversité par le nombre total de langues.

In [ ]:
language_size = []
for i in df['spoken_languages']:
    language_size.append((len(i.split(','))))
df = df.assign(nb_language = language_size)

In [ ]:
df.spoken_languages = df.spoken_languages.replace(to_replace=r'(^((?!English).)*$)', value=0, regex=True)
df.spoken_languages = df.spoken_languages.replace(to_replace=r'English', value=1, regex=True)

### Characters

On va garder le nombre de characters dans le film faire puis garder le pourcentage de homme et femme.
Une question va aussi se poser sur comment considérer l'ordre. Que cherche-t'on à mettre en évidence ?
A l'issue de cette transformation on aura déjà une information quand au ratio homme/femme. 
On pourrait chercher dans l'ordre, si il existe un désequilibre dans cet ordre (plus d'homme au début).
Mais cela me semble compliqué de réaliser cette tache sans produire trop de biais donc par simplicité on va garder la position de la première femme dans le casting et cette valeur sera plus ou moins représentatif de ce désequilibre. 
En réalité on a d'ailleur pas besoin de d'avoir l'ordre du casting puisque la liste des genres est déjà ordonnée. On peut donc retirer order dans le prétraitement.
Si il n'y a aucune femme alors on positionnera cette valeur comme étant à la dernière position


In [ ]:
gender_list = []
for i in df['characters_gender']:
    gender_list.extend(i.split(','))
print(Counter(gender_list))
print(len(Counter(gender_list))) # pas vraiment de valeur maquantes en réalité, c'est juste mauvaise utilisation du split

In [ ]:
# df.characters_gender = df.characters_gender.fillna(0)
i = 0
nb_cha = []
order_f = []
ratio_f = []
ratio_h = []
for line in df.characters_gender :
    liste = line.split(',')
    zero = liste.count('0')
    fem = liste.count('1')
    hom = liste.count('2')
    if fem > 0:
        order_f.append(liste.index('1'))
    else :
        order_f.append(fem + hom + zero)
    nb_cha.append(fem + hom + zero)
    if ((fem + hom + zero) > 0) :
        ratio_f.append(fem /(fem + hom + zero))
        ratio_h.append(hom /(fem + hom + zero))
    else :
        ratio_f.append(0)
        ratio_h.append(0)
    # print(zero,fem,hom)

df = df.assign(nb_ch = nb_cha)
df = df.assign(r_fem_ch = ratio_f)
df = df.assign(r_hom_ch = ratio_h)
df = df.assign(order_fem = order_f)
df.head()
# df.pop('C')

### Crew

In [ ]:
# df.characters_gender = df.characters_gender.fillna(0)
i = 0
nb_cha = []
ratio_f = []
ratio_h = []
for line in df.crew_gender :
    liste = line.split(',')
    zero = liste.count('0')
    fem = liste.count('1')
    hom = liste.count('2')
    nb_cha.append(fem + hom + zero)
    if ((fem + hom + zero) > 0) :
        ratio_f.append(fem /(fem + hom + zero))
        ratio_h.append(hom /(fem + hom + zero))
    else :
        ratio_f.append(0)
        ratio_h.append(0)
    # print(zero,fem,hom)

df = df.assign(nb_crew = nb_cha)
df = df.assign(r_fem_crew = ratio_f)
df = df.assign(r_hom_crew = ratio_h)
# df.pop('C')

In [ ]:
df.pop('characters_gender')
df.pop('crew_gender')

In [ ]:
df.head()

### Runtime

Il y a des valeurs de runtime abérante. Un runtime de 1256 est énorme mais peut correspondre à une grosse série. En revanche un runtime de 0 ne veut rien dire.

In [ ]:
print(df.runtime.min(), df.runtime.max())

In [ ]:
runtime_mean = df.runtime.mean()
df.runtime = df.runtime.replace(to_replace=0, value=runtime_mean)
df.runtime = df.runtime.fillna(runtime_mean)

### Données temporelles

On peut largement imaginer qu'il existe une influence du mois dans le succés d'un film. On va donc procéder à un prétraitement des données temporelles.

In [ ]:
# Converting the format of the date and creating new year, day, and month columns
# df.info()

df.release_date = pd.to_datetime(df.release_date)
df = df.drop(df[df.release_date.isnull()].index)
# df.drop[df["release_date"].isnull()]
df["release_year"] = pd.to_datetime(df["release_date"]).dt.year.astype(int)
# df["release_day"] = pd.to_datetime(df["release_date"]).dt.dayofweek.astype(int)
df["release_month"] = pd.to_datetime(df["release_date"]).dt.month.astype(int)


In [ ]:
print(df.release_year.max(),df.release_month.max(),df.release_year.min(),df.release_month.min())

In [ ]:
fig, ax = plt.subplots(3, 1, tight_layout=True)
plt.grid()

df.groupby('release_year')['revenue'].mean().plot(ax=ax[0], figsize=(10, 10), linewidth=3, color='green').set_title('Revenue over the Years', fontweight="bold")
df.groupby('release_month')['revenue'].mean().plot(ax=ax[1], figsize=(10, 10), linewidth=3, color='green').set_title('Revenue over the Months', fontweight="bold")


# df['Décennie'] = df['release_year'].apply(lambda x: str(x)[:3] + '0')
dbgroupby = df.groupby(['release_month',df['release_year'].apply(lambda x: str(x)[:3] + '0')])['revenue'].mean()
# dbgroupby.unstack().plot((ax=ax[2], figsize=(10,10))


# df_grouped = df.groupby(['nom', 'année']).sum()
dbgroupby = df.groupby(['release_month',df['release_year'].apply(lambda x: str(x)[:3] + '0')])['revenue'].mean().unstack().plot(ax=ax[2], figsize=(10, 10), linewidth=3).set_title('Revenue over the Months', fontweight="bold")

### One hot encoding

In [ ]:
months = ['jan', 'fev', 'mars', 'avril', 'mai', 'juin', 'juil', 'aout', 'sept', 'oct', 'nov', 'dec']

list_test = [months[int(i)-1] for i in df.release_month]
df = df.assign(release_month = list_test)

In [ ]:
df = pd.get_dummies(df, columns=['release_month'])

In [ ]:
df_genre = pd.Series(df.genres).str.get_dummies(sep=',')
df = pd.concat([df, df_genre], axis = 1) 

In [ ]:
df_genre.head()

In [ ]:
genre_size = []
for i in df['genres']:
    genre_size.append((len(i.split(','))))
df = df.assign(nb_genre = genre_size)

In [ ]:
df.pop('genres')
df.pop('release_date')

### Budget

Un bugdet et un revenu égal à 0 n'a pas de sens. On peut essayer de remplacer les variables par les plus proches voisins.\
Cette étapes aura nécessairement une influence importante sur notre résultat. \
On peut penser qu'un tel mécanisme peut conduire à influencer positivement l'influence du budget sur le revenu.

In [ ]:
print((df.revenue == 0).sum())
print((df.revenue.isna().sum()))
df.revenue.describe()

In [ ]:
df.pop('adult')
df.budget= df.budget.astype(int)

In [ ]:

df['budget'] = df['budget'].replace(0, np.nan)
df['revenue'] = df['revenue'].replace(0, np.nan)
# df.budget.isnull().sum()

In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=2)
df_filled = imputer.fit_transform(df)
df_filled = pd.DataFrame(df_filled, columns=df.columns)

In [ ]:
df_filled.to_csv('./data/diversity.csv', index=False)